In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model2_deepervgg
importlib.reload(kaggle); 
importlib.reload(model2_deepervgg);
from model2_deepervgg import Model, model_number

### Create datasets

In [2]:
complete_dataset, train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset(return_complete_set=True)

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

In [ ]:
kaggle.show_images(train_dataset, 9)

## Train new model

In [ ]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = 'Stacking_2'

model, history = kaggle.train_model(
    Model().build(None), prediction_train_set, prediction_valid_set, 
    epochs=10, valid_patience=4)

In [ ]:
# Save model
model.save(f'models/{model_number}/{model_name}_90')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

In [3]:
model = keras.models.load_model(f'models/{model_number}/WideResNet_2_73')

### Fine-tune

In [ ]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model_filepath(
    f'models/{model_number}/{model_name}',
    complete_dataset, valid_dataset, 
    epochs=4, learning_rate=1E-5)

In [ ]:
# Save fine-tuned model
fine_model_name = 'Stacking_2'
fine_model.save(f'models/{model_number}/{fine_model_name}')

### Evaluate

In [ ]:
model_name = ''
# model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(prediction_valid_set), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = 'Stacking_2_90'
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}', x_test=prediction_test_set)

In [ ]:
kaggle.generate_test_pred(fine_model, f'models/{model_number}/{model_name}_test_pred.csv')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), prediction_train_set, prediction_valid_set, 
    model_number, 'hypertuner2021-12-08', trials=60, tuner_type='bayesian',
    epochs=15, valid_patience=4)

In [3]:
import importlib
import kaggle_functions as kaggle
import model5_wideresnet
importlib.reload(model5_wideresnet);
from model5_wideresnet import Model
model_number = 'model10'

tuner = kaggle.load_hypertuner(Model(), model_number, 'hypertuner2021-12-03')

INFO:tensorflow:Reloading Oracle from existing project models/model10\hypertuner2021-12-03\oracle.json
INFO:tensorflow:Reloading Tuner from models/model10\hypertuner2021-12-03\tuner0.json
Results summary
Results in models/model10\hypertuner2021-12-03
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_dropout: 0.5
k: 9
Score: 0.8048780560493469
Trial summary
Hyperparameters:
conv_dropout: 0.3
k: 9
Score: 0.8015138506889343
Trial summary
Hyperparameters:
conv_dropout: 0.5
k: 6
Score: 0.7211942672729492


In [23]:
model = tuner.get_best_models(1)[0]